In [41]:
!pip install dash pandas dash pyngrok

In [ ]:
import gdown

# Replace with your Google Drive file ID
file_id = "1KqFpEyCFPyqFdqL9VWBvyULcycIcB_St"
url = f"https://drive.google.com/file/d/1KqFpEyCFPyqFdqL9VWBvyULcycIcB_St/view?usp=drive_link"
output = "Blast21-24.csv"
gdown.download(url, output, quiet=False)

In [42]:
import pandas as pd
import numpy as np

df = pd.read_csv('Blast21-24.csv' , low_memory=False)

# Create the binary column 'out_due_to_bowler' using the provided logic for 'Wicket'
df['out_due_to_bowler'] = df['Wicket'].notna() & ~df['Wicket'].isin(
    ['Run Out', 'Obstruction', 'Retired - Not Out', 'Handled Ball', 'Absent', 'Retired Hurt']
)

# Initialize dictionaries to store results for strike rate and wicket average
best_bowler_type_strike_rate = {}
best_bowler_type_wicket_average = {}

# Loop through each batter
for batter in df['Batter'].unique():
    # Create an empty dictionary to store statistics for each bowler type
    batter_stats_strike_rate = {}
    batter_stats_wicket_average = {}

    # Loop through each unique bowler type for the specific batter
    for b in df[df['Batter'] == batter]['Bowler Type'].unique():
        relevant_data = df[(df['Batter'] == batter) & (df['Bowler Type'] == b)]

        total_runs = relevant_data['Runs'].sum()
        total_balls = relevant_data['Cumulative Batter Balls'].shape[0]  # Using 'Cumulative Batter Balls' column
        wickets = relevant_data['out_due_to_bowler'].sum()

        if total_balls > 0:
            runs_per_ball = total_runs / total_balls
            # Convert runs per ball to strike rate by multiplying by 100
            strike_rate = runs_per_ball * 100
        else:
            strike_rate = 8888

        if wickets > 0:
            wicket_average = total_runs / wickets
        else:
            wicket_average = 8888

        # Store statistics for each bowler type for the current batter
        batter_stats_strike_rate[b] = (strike_rate, total_balls)
        batter_stats_wicket_average[b] = (wicket_average, wickets)

    # Sort bowler types by strike rate and wicket average
    sorted_strike_rate = {k: v for k, v in sorted(batter_stats_strike_rate.items(), key=lambda item: item[1][0])}
    sorted_wicket_average = {k: v for k, v in sorted(batter_stats_wicket_average.items(), key=lambda item: item[1][0], reverse=True)}

    # Store the sorted statistics for each batter
    best_bowler_type_strike_rate[batter] = sorted_strike_rate
    best_bowler_type_wicket_average[batter] = sorted_wicket_average

In [43]:
# Print the sorted statistics for each batter
for batter in best_bowler_type_strike_rate:
    print(f"Best bowler types against {batter} (by strike rate):")
    for b, (strike_rate, balls_faced) in best_bowler_type_strike_rate[batter].items():
        print(f"{b}: {strike_rate} (Balls faced: {balls_faced})")

Best bowler types against G Roelofsen (by strike rate):
RM: 57.14285714285714 (Balls faced: 7)
ROB: 108.69565217391303 (Balls faced: 23)
LFM: 111.11111111111111 (Balls faced: 36)
RFM: 143.2 (Balls faced: 125)
LOB: 150.0 (Balls faced: 6)
RLB: 163.63636363636365 (Balls faced: 11)
Best bowler types against CDJ Dent (by strike rate):
LF: 50.0 (Balls faced: 12)
RLB: 84.21052631578947 (Balls faced: 19)
LLB: 107.6923076923077 (Balls faced: 13)
ROB: 110.00000000000001 (Balls faced: 40)
LOB: 166.66666666666669 (Balls faced: 15)
RFM: 170.92198581560282 (Balls faced: 141)
LFM: 178.0487804878049 (Balls faced: 41)
RM: 233.33333333333334 (Balls faced: 12)
Best bowler types against MAH Hammond (by strike rate):
LF: 0.0 (Balls faced: 2)
LM: 41.66666666666667 (Balls faced: 12)
RF: 45.45454545454545 (Balls faced: 11)
RM: 92.10526315789474 (Balls faced: 76)
ROB: 98.75 (Balls faced: 80)
LFM: 107.75862068965519 (Balls faced: 116)
RFM: 140.81196581196582 (Balls faced: 468)
RLB: 142.22222222222223 (Balls fac

In [44]:
data_to_append = []

for batter in best_bowler_type_strike_rate:
    for b, (strike_rate, balls_faced) in best_bowler_type_strike_rate[batter].items():
        data_to_append.append({"Batter": batter, "BowlerType": b, "StrikeRate": strike_rate, "BallsFaced": balls_faced})

results_df = pd.DataFrame(data_to_append)

results_df.to_csv("strike_rate_against_bowlers.csv", index=False)
print("Output has been saved to 'strike_rate_against_bowlers.csv'")

Output has been saved to 'strike_rate_against_bowlers.csv'


In [45]:
for batter in best_bowler_type_wicket_average:
    print(f"Best bowler types against {batter} (by bowling average):")
    for b, (wicket_avg, wickets) in sorted(best_bowler_type_wicket_average[batter].items(), key=lambda x: x[1][0]):
        print(f"{b}: {wicket_avg} (Dismissals: {wickets})")

Best bowler types against G Roelofsen (by bowling average):
RM: 4.0 (Dismissals: 1)
RLB: 9.0 (Dismissals: 2)
LOB: 9.0 (Dismissals: 1)
LFM: 20.0 (Dismissals: 2)
RFM: 35.8 (Dismissals: 5)
ROB: 8888 (Dismissals: 0)
Best bowler types against CDJ Dent (by bowling average):
LF: 3.0 (Dismissals: 2)
RLB: 5.333333333333333 (Dismissals: 3)
LLB: 14.0 (Dismissals: 1)
ROB: 22.0 (Dismissals: 2)
RFM: 30.125 (Dismissals: 8)
LFM: 73.0 (Dismissals: 1)
RM: 8888 (Dismissals: 0)
LOB: 8888 (Dismissals: 0)
Best bowler types against MAH Hammond (by bowling average):
LF: 0.0 (Dismissals: 1)
RF: 5.0 (Dismissals: 1)
LM: 5.0 (Dismissals: 1)
ROB: 7.9 (Dismissals: 10)
RLB: 16.0 (Dismissals: 4)
LFM: 17.857142857142858 (Dismissals: 7)
LOB: 21.8 (Dismissals: 5)
RFM: 31.38095238095238 (Dismissals: 21)
RM: 35.0 (Dismissals: 2)
Best bowler types against OJ Price (by bowling average):
LFM: 1.3333333333333333 (Dismissals: 3)
RM: 12.0 (Dismissals: 1)
RLB: 13.333333333333334 (Dismissals: 3)
RFM: 27.25 (Dismissals: 4)
LOB: 35

In [46]:
# Create a list to store the data
data = []

# Populate the list with the sorted statistics
for batter in best_bowler_type_wicket_average:
    for b, (wicket_avg, wickets) in sorted(best_bowler_type_wicket_average[batter].items(), key=lambda x: x[1][0], reverse=True):
        data.append({
            "Batter": batter,
            "BowlerType": b,
            "WicketAverage": wicket_avg,
            "Dismissals": wickets
        })
        print(f"{b}: {wicket_avg} (Dismissals: {wickets})")

# Convert the list of dictionaries to a DataFrame
results_wicket_avg_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
results_wicket_avg_df.to_csv("bowling_average.csv", index=False)

print("Wicket average output has been saved to 'bowling_average.csv'")

ROB: 8888 (Dismissals: 0)
RFM: 35.8 (Dismissals: 5)
LFM: 20.0 (Dismissals: 2)
RLB: 9.0 (Dismissals: 2)
LOB: 9.0 (Dismissals: 1)
RM: 4.0 (Dismissals: 1)
RM: 8888 (Dismissals: 0)
LOB: 8888 (Dismissals: 0)
LFM: 73.0 (Dismissals: 1)
RFM: 30.125 (Dismissals: 8)
ROB: 22.0 (Dismissals: 2)
LLB: 14.0 (Dismissals: 1)
RLB: 5.333333333333333 (Dismissals: 3)
LF: 3.0 (Dismissals: 2)
RM: 35.0 (Dismissals: 2)
RFM: 31.38095238095238 (Dismissals: 21)
LOB: 21.8 (Dismissals: 5)
LFM: 17.857142857142858 (Dismissals: 7)
RLB: 16.0 (Dismissals: 4)
ROB: 7.9 (Dismissals: 10)
RF: 5.0 (Dismissals: 1)
LM: 5.0 (Dismissals: 1)
LF: 0.0 (Dismissals: 1)
LLB: 8888 (Dismissals: 0)
LF: 8888 (Dismissals: 0)
ROB: 44.0 (Dismissals: 1)
LOB: 35.5 (Dismissals: 2)
RFM: 27.25 (Dismissals: 4)
RLB: 13.333333333333334 (Dismissals: 3)
RM: 12.0 (Dismissals: 1)
LFM: 1.3333333333333333 (Dismissals: 3)
RF: 8888 (Dismissals: 0)
RLB: 82.0 (Dismissals: 1)
LF: 44.0 (Dismissals: 1)
LOB: 29.0 (Dismissals: 4)
RM: 26.5 (Dismissals: 2)
RFM: 17.125

In [47]:
# Initialize a dictionary to store dismissal rates
dismissal_rate = {}

# Loop through each batter
for batter in df['Batter'].unique():
    dismissal_rate[batter] = {}

    # Loop through each unique bowler type for the specific batter
    for b in df[df['Batter'] == batter]['Bowler Type'].unique():
        relevant_data = df[(df['Batter'] == batter) & (df['Bowler Type'] == b)]
        total_balls_faced = relevant_data.shape[0]  # Since 'Cumulative Batter Balls' gives us total balls faced
        dismissals = relevant_data['out_due_to_bowler'].sum()

        if dismissals > 0:
            # Calculate dismissal rate (total_balls_faced / dismissals)
            rate = total_balls_faced / dismissals
        else:
            rate = 8888  # No dismissals, so dismissal rate is infinite

        dismissal_rate[batter][b] = (rate, dismissals)

# Output the dismissal rates in a sorted manner for better readability
for batter, rates in dismissal_rate.items():
    print(f"Balls per dismissal for {batter} against:")
    # Sort bowler types by dismissal rate
    sorted_rates = {k: v for k, v in sorted(rates.items(), key=lambda item: item[1])}
    for b, (rate, wickets) in sorted_rates.items():
        print(f"{b}: {rate} (Dismissals: {wickets})")

Balls per dismissal for G Roelofsen against:
RLB: 5.5 (Dismissals: 2)
LOB: 6.0 (Dismissals: 1)
RM: 7.0 (Dismissals: 1)
LFM: 18.0 (Dismissals: 2)
RFM: 25.0 (Dismissals: 5)
ROB: 8888 (Dismissals: 0)
Balls per dismissal for CDJ Dent against:
LF: 6.0 (Dismissals: 2)
RLB: 6.333333333333333 (Dismissals: 3)
LLB: 13.0 (Dismissals: 1)
RFM: 17.625 (Dismissals: 8)
ROB: 20.0 (Dismissals: 2)
LFM: 41.0 (Dismissals: 1)
RM: 8888 (Dismissals: 0)
LOB: 8888 (Dismissals: 0)
Balls per dismissal for MAH Hammond against:
LF: 2.0 (Dismissals: 1)
ROB: 8.0 (Dismissals: 10)
RF: 11.0 (Dismissals: 1)
RLB: 11.25 (Dismissals: 4)
LM: 12.0 (Dismissals: 1)
LOB: 14.8 (Dismissals: 5)
LFM: 16.571428571428573 (Dismissals: 7)
RFM: 22.285714285714285 (Dismissals: 21)
RM: 38.0 (Dismissals: 2)
Balls per dismissal for OJ Price against:
LFM: 3.3333333333333335 (Dismissals: 3)
RM: 12.0 (Dismissals: 1)
RLB: 15.0 (Dismissals: 3)
LOB: 20.5 (Dismissals: 2)
RFM: 22.0 (Dismissals: 4)
ROB: 35.0 (Dismissals: 1)
LLB: 8888 (Dismissals: 0)


In [48]:
# Create a list to store the data
data = []

# Populate the list with the dismissal rates
for batter, rates in dismissal_rate.items():
    for b, (rate, dismissals) in rates.items():
        data.append({
            "Batter": batter,
            "BowlerType": b,
            "DismissalRate": rate,
            "Dismissals": dismissals
        })
        print(f"{b}: {rate} (Dismissals: {dismissals})")

# Convert the list of dictionaries to a DataFrame
dismissal_rate_df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
dismissal_rate_df.to_csv("dismissal_rate_against_bowlers.csv", index=False)

print("Dismissal rate output has been saved to 'dismissal_rate_against_bowlers.csv'")

ROB: 8888 (Dismissals: 0)
RFM: 25.0 (Dismissals: 5)
LFM: 18.0 (Dismissals: 2)
RLB: 5.5 (Dismissals: 2)
LOB: 6.0 (Dismissals: 1)
RM: 7.0 (Dismissals: 1)
ROB: 20.0 (Dismissals: 2)
RFM: 17.625 (Dismissals: 8)
RM: 8888 (Dismissals: 0)
LOB: 8888 (Dismissals: 0)
LFM: 41.0 (Dismissals: 1)
RLB: 6.333333333333333 (Dismissals: 3)
LF: 6.0 (Dismissals: 2)
LLB: 13.0 (Dismissals: 1)
RFM: 22.285714285714285 (Dismissals: 21)
ROB: 8.0 (Dismissals: 10)
RM: 38.0 (Dismissals: 2)
LFM: 16.571428571428573 (Dismissals: 7)
RLB: 11.25 (Dismissals: 4)
LOB: 14.8 (Dismissals: 5)
LF: 2.0 (Dismissals: 1)
RF: 11.0 (Dismissals: 1)
LM: 12.0 (Dismissals: 1)
ROB: 35.0 (Dismissals: 1)
RFM: 22.0 (Dismissals: 4)
LOB: 20.5 (Dismissals: 2)
LFM: 3.3333333333333335 (Dismissals: 3)
RLB: 15.0 (Dismissals: 3)
RM: 12.0 (Dismissals: 1)
LLB: 8888 (Dismissals: 0)
LF: 8888 (Dismissals: 0)
ROB: 11.8 (Dismissals: 5)
RFM: 14.1875 (Dismissals: 16)
LOB: 23.0 (Dismissals: 4)
RM: 19.0 (Dismissals: 2)
LFM: 7.5 (Dismissals: 6)
RLB: 47.0 (Dismis

In [49]:
dismissal_rate_df = pd.read_csv('dismissal_rate_against_bowlers.csv')
strike_rate_df = pd.read_csv('strike_rate_against_bowlers.csv')

# Replace 8888 with NaN
strike_rate_df['StrikeRate'] = strike_rate_df['StrikeRate'].replace(8888, np.nan)
dismissal_rate_df['DismissalRate'] = dismissal_rate_df['DismissalRate'].replace(8888, np.nan)

def calculate_effectiveness(batters, bowler_types, weightages):
    """
    Calculate the combined effectiveness score with separate SR and DR weightages for each batter.

    :param batters: List of batter names on the crease.
    :param bowler_types: List of available bowler types.
    :param weightages: Dictionary specifying SR and DR weightages for each batter.
    """
    effectiveness_scores = {}
    for bowler_type in bowler_types:
        total_score = 0
        for batter in batters:
            # Retrieve specified weightages for the current batter
            weightage_sr, weightage_dr = weightages[batter]['sr'], weightages[batter]['dr']

            # Strike Rate calculation
            sr_data = strike_rate_df[(strike_rate_df['Batter'] == batter) & (strike_rate_df['BowlerType'] == bowler_type)]
            sr = sr_data['StrikeRate'].mean() if not sr_data.empty else np.nan

            # Dismissal Rate calculation
            dr_data = dismissal_rate_df[(dismissal_rate_df['Batter'] == batter) & (dismissal_rate_df['BowlerType'] == bowler_type)]
            dr = dr_data['DismissalRate'].mean() if not dr_data.empty else np.nan

            # Initialize extra points
            extra_points = 0

            # Check if DR is NaN and add extra points if so
            if np.isnan(dr):
                extra_points += 30

            # Check if SR is NaN and add extra points if so
            if np.isnan(sr):
                extra_points += 30

            score = (weightage_sr * sr if not np.isnan(sr) else 0) + (weightage_dr * dr if not np.isnan(dr) else 0) + extra_points
            total_score += score

        effectiveness_scores[bowler_type] = total_score / len(batters)  # Average score for fairness

    # Sort bowler types by effectiveness score in ascending order
    sorted_effectiveness = dict(sorted(effectiveness_scores.items(), key=lambda item: item[1]))

    return sorted_effectiveness

# Specify weightages for SR and DR separately for both the batters
weightages = {
    'CA Lynn': {'sr': 0.25, 'dr': 0.25},
    'RS Vasconcelos': {'sr': 0.25, 'dr': 0.25}
}

batters_on_crease = ['CA Lynn', 'RS Vasconcelos']
available_bowler_types = ['RF', 'LOB', 'LFM', 'RFM']

# Calculate and print the sorted effectiveness scores
sorted_scores = calculate_effectiveness(batters_on_crease, available_bowler_types, weightages)
print("Effectiveness Rating (lower the better for deployment):")
for bowler_type, score in sorted_scores.items():
    print(f"{bowler_type}: {score}")

Effectiveness Rating (lower the better for deployment):
LFM: 25.744502314814817
RFM: 45.11565158697735
LOB: 49.23863636363636
RF: 53.14484126984127


In [50]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
from pyngrok import ngrok

# Authentication Token
ngrok.set_auth_token('2eH1AAVtJ7ZA9ukIOKr7DZWoOaZ_4wPkiQgGCer3xUumKjRQr')

# Loading the data
dismissal_rate_df = pd.read_csv('dismissal_rate_against_bowlers.csv')
strike_rate_df = pd.read_csv('strike_rate_against_bowlers.csv')

# Replace 8888 with NaN
strike_rate_df['StrikeRate'] = strike_rate_df['StrikeRate'].replace(8888, np.nan)
dismissal_rate_df['DismissalRate'] = dismissal_rate_df['DismissalRate'].replace(8888, np.nan)

app = Dash(__name__)

# App layout
app.layout = html.Div([
    dcc.Input(id='batters-input', type='text', value='CA Lynn,RS Vasconcelos', style={'width': '300px'}),
    dcc.Input(id='bowler-types-input', type='text', value='RF,LOB,RFM,LFM', style={'width': '300px'}),

    html.Br(),
    html.Label('Batter 1 SR Weightage:'),
    dcc.Input(id='batter1-sr-weightage', type='number', value=0.25, min=0, max=1, step=0.01),

    html.Label('Batter 1 DR Weightage:'),
    dcc.Input(id='batter1-dr-weightage', type='number', value=0.25, min=0, max=1, step=0.01),

    html.Br(),
    html.Label('Batter 2 SR Weightage:'),
    dcc.Input(id='batter2-sr-weightage', type='number', value=0.25, min=0, max=1, step=0.01),

    html.Label('Batter 2 DR Weightage:'),
    dcc.Input(id='batter2-dr-weightage', type='number', value=0.25, min=0, max=1, step=0.01),

    html.Br(),
    html.Button('Submit', id='submit-val', n_clicks=0),
    html.Div(id='container-div')
])

@app.callback(
    Output('container-div', 'children'),
    [Input('submit-val', 'n_clicks')],
    [State('batters-input', 'value'),
     State('bowler-types-input', 'value'),
     State('batter1-sr-weightage', 'value'),
     State('batter1-dr-weightage', 'value'),
     State('batter2-sr-weightage', 'value'),
     State('batter2-dr-weightage', 'value')]
)
def update_output(n_clicks, batters_input, bowler_types_input,
                  batter1_sr_weightage, batter1_dr_weightage,
                  batter2_sr_weightage, batter2_dr_weightage):
    if n_clicks > 0:
        batters = batters_input.split(',')
        bowler_types = bowler_types_input.split(',')

        weightages = {
            batters[0]: {'sr': float(batter1_sr_weightage), 'dr': float(batter1_dr_weightage)},
            batters[1]: {'sr': float(batter2_sr_weightage), 'dr': float(batter2_dr_weightage)}
        }

        sorted_scores = calculate_effectiveness(batters, bowler_types, weightages)
        children = [html.Div(f'{bowler_type}: {score}') for bowler_type, score in sorted_scores.items()]
        return children
    return "Enter details and press submit."

def calculate_effectiveness(batters, bowler_types, weightages):
    effectiveness_scores = {}
    for bowler_type in bowler_types:
        total_score = 0
        for batter in batters:
            weightage_sr, weightage_dr = weightages[batter]['sr'], weightages[batter]['dr']

            sr_data = strike_rate_df[(strike_rate_df['Batter'] == batter) & (strike_rate_df['BowlerType'] == bowler_type)]
            sr = sr_data['StrikeRate'].mean() if not sr_data.empty else np.nan

            dr_data = dismissal_rate_df[(dismissal_rate_df['Batter'] == batter) & (dismissal_rate_df['BowlerType'] == bowler_type)]
            dr = dr_data['DismissalRate'].mean() if not dr_data.empty else np.nan

            # Calculate combined score (lower is better)
            # Initialize extra points
            extra_points = 0

            # Check if DR is NaN and add extra points if so
            if np.isnan(dr):
                extra_points += 30

            # Check if SR is NaN and add extra points if so
            if np.isnan(sr):
                extra_points += 30

            score = (weightage_sr * sr if not np.isnan(sr) else 0) + (weightage_dr * dr if not np.isnan(dr) else 0) + extra_points
            total_score += score

        effectiveness_scores[bowler_type] = total_score / len(batters)

    sorted_effectiveness = dict(sorted(effectiveness_scores.items(), key=lambda item: item[1]))
    return sorted_effectiveness

reserved_domain = 'darling-lacewing-stirred.ngrok-free.app'  # Replace with your domain
ngrok_tunnel = ngrok.connect(addr=8050, hostname=reserved_domain, bind_tls=True)

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True, port=8050)

<IPython.core.display.Javascript object>

In [51]:
print('Public URL:', ngrok_tunnel.public_url)

Public URL: https://darling-lacewing-stirred.ngrok-free.app
